# Análisis y Curación de Datos

### Motivación


- La idea de este práctico es realizar un análisis, limpieza y transformación de los datos para su posterior utilización en la resolución de un problema específico junto con la ayuda de modelos de aprendizaje automático. El problema que se va a abordar en el siguiente práctico y para lo cual se va a manipular el conjunto de datos es estimar el monto de la compra de un usuario.

### Consignas:


0. Cargar el archivo *dirty_retail_sales.zip* como un dataframe de Pandas.


1. Transformar los valores de la columna del precio de compra para que sea un valor numérico en lugar de una cadena de caracteres donde los últimos dos digitos corresponden a la parte decimal.


2. Separar los valores de la columna *Gender_Age_Marital_Status* en tres nuevas columnas para luego eliminar la columna original.


3. Dado un *Product_ID* encontramos distintos valores de compra (*Purchase*) asociados. Nos gustaría tener un único valor de compra representativo. Analizar cual sería un buen estimador y posteriormente imputar ese valor para cada *Product_ID* en particular. 


4. Consistencia de los datos: En este dataset cada registro tiene información relacionada a un usuario y a un producto adquirido por el mismo. Bajo la hipótesis de que los valores de *User_ID* y *Product_ID* identifican unívocamente a cada usuario y producto se busca chequear la consistencia de los demas valores que los caracterizan.

    En concreto, se debe validar que para cada *User_ID* los atributos *Gender*, *Age*, *Occupation*, *City_Category*, *Stay_In_Current_City_Years* y *Marital_Status* son los mismos por cada registro donde ocurren. De manera análoga, en el caso de los productos, para cada *Product_ID* validar que los principales atributos que lo describen: *Product_Category_1*, *Product_Category_2* y *Product_Category_3* son los mismos en cada registro.


5. Tanto en la columna de *Product_Category_2* como *Product_Category_3* encontramos valores faltantes. Proponer una alternativa para tratar estos valores nulos ya que la opción de eliminar los registros que poseen valores faltantes provocaría una gran perdida de información por lo que no es viable.


6. Investigar por qué y cómo se realiza un *encoding* de variables categóricas para su posterior utilización en un modelo de Machine Learning. Un post interesante es el siguiente: https://towardsdatascience.com/understanding-feature-engineering-part-2-categorical-data-f54324193e63

   En particular, generar *features* a partir de cada valor categórico del dataset aplicando la estrategia tradicional de *one-hot encoding*.

In [1]:
import pandas as pd
import seaborn
import matplotlib as plt
import numpy as np
import plotly.plotly as py
import plotly.figure_factory as ff

In [2]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import Imputer

In [3]:
data = pd.read_csv('dirty_retail_sales.csv')
data[:5]

,User_ID,Product_ID,Occupation,City_Category,Stay_In_Current_City_Years,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,Gender_Age_Marital_Status
0,1000001,P00069042,10,A,2,3,NaN,NaN,8370,F_0-17_0
1,1000001,P00248942,10,A,2,1,6.0,14.0,15200,F_0-17_0
2,1000001,P00087842,10,A,2,12,NaN,NaN,1422,F_0-17_0
3,1000001,P00085442,10,A,2,12,14.0,NaN,1057,F_0-17_0
4,1000002,P00285442,16,C,4+,8,NaN,NaN,7969,M_55+_0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537577 entries, 0 to 537576
Data columns (total 10 columns):
User_ID                       537577 non-null int64
Product_ID                    537577 non-null object
Occupation                    537577 non-null int64
City_Category                 537577 non-null object
Stay_In_Current_City_Years    537577 non-null object
Product_Category_1            537577 non-null int64
Product_Category_2            370591 non-null float64
Product_Category_3            164278 non-null float64
Purchase                      537577 non-null int64
Gender_Age_Marital_Status     537577 non-null object
dtypes: float64(2), int64(4), object(4)
memory usage: 41.0+ MB


**1) Transformación de los valores de la columna "Purchase" en valores numéricos.**

In [5]:
data_1 = data['Purchase'].astype(float)/100
data_1 = pd.DataFrame(data_1)
data_1.rename(columns={'Purchase':'Purchase_1'}, inplace=True)
data_v = data.join(data_1)

In [6]:
data_vf = data_v.drop(['Purchase'], axis=1)

**2) Separar los valores de la columna Gender_Age_Marital_Status en tres nuevas columnas para luego eliminar la columna original.**

In [7]:
clean = data_vf['Gender_Age_Marital_Status'].str.split(pat = "_", expand=True)
clean.rename(columns={0:'Gender', 1:'Age', 2:'Marital_Status'}, inplace=True)

In [8]:
clean_1 = data_vf.join(clean)

In [9]:
clean_vf = clean_1.drop(['Gender_Age_Marital_Status'], axis=1)

**3) Contar con un único valor de compras representativo para cada Product_ID.**

In [10]:
prod = pd.DataFrame(clean_vf, columns=['Product_ID', 'Purchase_1'])  

Calculamos la Media de los valores de las compras

In [11]:
prod_mean = pd.DataFrame(prod.groupby('Product_ID', as_index=False).mean().rename(columns={'Purchase_1':'Purchase_Media'}))

Calculamos la Mediana de los valores de las compras

In [12]:
prod_median = pd.DataFrame(prod.groupby('Product_ID', as_index=False).median()
                           .rename(columns={'Purchase_1':'Purchase_Mediana'}))

Anexamos las nuevas columnas que sirven como candidato para reemplazar la columna Purchase_1

In [13]:
purchase = pd.merge(prod_mean, prod_median, how='left', on='Product_ID')

In [14]:
purchase_1 = pd.merge(clean_vf, purchase, how='left', on='Product_ID')

In [15]:
dataset = purchase_1.drop(['Purchase_1'], axis=1)

Dataset por lo tanto quedaria como la base que contiene todos los valores corregidos y modificados, lista para continuar con el análisis.

**4) Validar los datos de User_ID y Product_ID.**

*Primero validamos los datos para cada User_ID*

In [16]:
user = pd.DataFrame(dataset, columns=['User_ID', 'Gender', 'Age', 'Occupation', 'Stay_In_Current_City_Years', 'Marital_Status'])

In [17]:
user_dupl = user[user.User_ID.duplicated(keep=False)]
user_dupl

,User_ID,Gender,Age,Occupation,Stay_In_Current_City_Years,Marital_Status
0,1000001,F,0-17,10,2,0
1,1000001,F,0-17,10,2,0
2,1000001,F,0-17,10,2,0
3,1000001,F,0-17,10,2,0
4,1000002,M,55+,16,4+,0
5,1000003,M,26-35,15,3,0
6,1000004,M,46-50,7,2,1
7,1000004,M,46-50,7,2,1
8,1000004,M,46-50,7,2,1
9,1000005,M,26-35,20,1,1


Tal como se observa en el DataFrame previo, todo parece indicar que cada User_ID que aparece más de una vez presenta cualidades singulares y no contradictorias. Para confirmar lo dicho, se cruzan los datos duplicados que podrían llegar a existir. En el caso que un mismo usuario presente datos distintos, el registro no se anularía y aparecería en el siguiente cuadro. 

In [18]:
user_dupl.drop_duplicates()[user_dupl.drop_duplicates().User_ID.duplicated()]

,User_ID,Gender,Age,Occupation,Stay_In_Current_City_Years,Marital_Status


*Segundo validamos que existen datos únicos para cada Product_ID*

In [19]:
prod = pd.DataFrame(dataset, columns=['Product_ID', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3'])

In [20]:
prod_dupl = prod[prod.Product_ID.duplicated(keep=False)].sort_values(by="Product_ID")
prod_dupl

,Product_ID,Product_Category_1,Product_Category_2,Product_Category_3
36015,P00000142,3,4.0,5.0
273508,P00000142,3,4.0,5.0
169172,P00000142,3,4.0,5.0
303130,P00000142,3,4.0,5.0
106598,P00000142,3,4.0,5.0
255473,P00000142,3,4.0,5.0
45940,P00000142,3,4.0,5.0
415524,P00000142,3,4.0,5.0
335497,P00000142,3,4.0,5.0
335518,P00000142,3,4.0,5.0


In [21]:
prod_dupl.drop_duplicates()[prod_dupl.drop_duplicates().Product_ID.duplicated()]

,Product_ID,Product_Category_1,Product_Category_2,Product_Category_3


Al igual que sucede con el User_ID, en el caso de Product_ID no se repiten los registros para para ID

**5) Alernativa para reemplazar los valores faltantes de Product_Category_2 y Product_Category_3.**

In [22]:
null = pd.DataFrame(dataset, columns=['Product_ID', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3', 'Purchase_Media', 'Purchase_Mediana'])

Primero debemos detectar la cantidad de valores faltantes en la muestra

In [23]:
null_count = null.isnull().sum()
null_count

Product_ID                 0
Product_Category_1         0
Product_Category_2    166986
Product_Category_3    373299
Purchase_Media             0
Purchase_Mediana           0
dtype: int64

*Debemos relativizar el porcentaje de valores nulos en cada categoría.*

In [24]:
print('Si borramos los valores nulos presentes en la categoría 2, quedaría el', len(null.dropna(subset=['Product_Category_2']))/len(null),' del total de registros en la muestra.')

Si borramos los valores nulos presentes en la categoría 2, quedaría el 0.6893728712351905  del total de registros en la muestra.


In [25]:
print('Si borramos los valores nulos presentes en la categoría 3, quedaría el', len(null.dropna(subset=['Product_Category_3']))/len(null),' del total de registros en la muestra.')

Si borramos los valores nulos presentes en la categoría 3, quedaría el 0.3055897108693266  del total de registros en la muestra.


In [26]:
prod_2 = 1 - len(null.dropna(subset=['Product_Category_2']))/len(null)
prod_3 = 1 - len(null.dropna(subset=['Product_Category_3']))/len(null)

print('La cantidad de valores nulos en la categoría 2 representan el',prod_2, 'del total de registros\nLa cantidad de valores nulos en la categoría 3 representan el', prod_3, 'del total de registros')

La cantidad de valores nulos en la categoría 2 representan el 0.31062712876480947 del total de registros
La cantidad de valores nulos en la categoría 3 representan el 0.6944102891306734 del total de registros


La presencia de valores nulos en la muestra es elevada, por lo tanto eliminarlos no es una buena opción dado que se perdería más del 50% del total de registros de la muestra.

No obstante, como una primer medida para rescatar un valor más concreto de los valores faltantes, se van a completar con 0 los valores nulos.

In [35]:
dataset = dataset.fillna(0)

In [43]:
dataset.isnull().sum()

User_ID                       0
Product_ID                    0
Occupation                    0
City_Category                 0
Stay_In_Current_City_Years    0
Product_Category_1            0
Product_Category_2            0
Product_Category_3            0
Gender                        0
Age                           0
Marital_Status                0
Purchase_Media                0
Purchase_Mediana              0
dtype: int64

In [44]:
dataset.shape

(537577, 13)

**6) Generar features a partir de cada valor categórico del dataset aplicando la estrategia tradicional de one-hot encoding.**

In [46]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [47]:
dataset.info('object')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 537577 entries, 0 to 537576
Data columns (total 13 columns):
User_ID                       537577 non-null int64
Product_ID                    537577 non-null object
Occupation                    537577 non-null int64
City_Category                 537577 non-null object
Stay_In_Current_City_Years    537577 non-null object
Product_Category_1            537577 non-null int64
Product_Category_2            537577 non-null float64
Product_Category_3            537577 non-null float64
Gender                        537577 non-null object
Age                           537577 non-null object
Marital_Status                537577 non-null object
Purchase_Media                537577 non-null float64
Purchase_Mediana              537577 non-null float64
dtypes: float64(4), int64(3), object(6)
memory usage: 57.4+ MB


In [48]:
encode = pd.DataFrame(dataset, columns=['Product_ID', 'City_Category', 'Gender', 'Age'])

In [49]:
# Codificación para el feature "City_Category"

cit_lab = LabelEncoder()
cit_labels = cit_lab.fit_transform(encode['City_Category'])
encode['cit_label'] = cit_labels

# Codificación para el feature "Gender"

gen_lab = LabelEncoder()
gen_labels = gen_lab.fit_transform(encode['Gender'])
encode['gen_Label'] = gen_labels

# Codificación para el feature "Age"

age_lab = LabelEncoder()
age_labels = age_lab.fit_transform(encode['Age'])
encode['age_Label'] = age_labels

Ahora aplicamos el one-hot encoder.

In [50]:
# Primero para la variable "City_Category"

cit_ohe = OneHotEncoder()
cit_feature_arr = cit_ohe.fit_transform(
                              encode[['City_Category']]).toarray()
cit_feature_labels = list(cit_lab.classes_)
cit_features = pd.DataFrame(cit_feature_arr, 
                            columns=cit_feature_labels)

# Segundo para la variable "Gender"

gen_ohe = OneHotEncoder()
gen_feature_arr = gen_ohe.fit_transform(
                              encode[['Gender']]).toarray()
gen_feature_labels = list(gen_lab.classes_)
gen_features = pd.DataFrame(gen_feature_arr, 
                            columns=gen_feature_labels)

# Tercero para la variable "Age"

age_ohe = OneHotEncoder()
age_feature_arr = age_ohe.fit_transform(
                              encode[['Age']]).toarray()
age_feature_labels = list(age_lab.classes_)
age_features = pd.DataFrame(age_feature_arr, 
                            columns=age_feature_labels)

In [51]:
dataset_ohe = pd.concat([dataset, cit_features, gen_features, age_features], axis=1)
dataset_ohe

,User_ID,Product_ID,Occupation,City_Category,Stay_In_Current_City_Years,Product_Category_1,Product_Category_2,Product_Category_3,Gender,Age,...,C,F,M,0-17,18-25,26-35,36-45,46-50,51-55,55+
0,1000001,P00069042,10,A,2,3,0.0,0.0,F,0-17,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1000001,P00248942,10,A,2,1,6.0,14.0,F,0-17,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1000001,P00087842,10,A,2,12,0.0,0.0,F,0-17,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1000001,P00085442,10,A,2,12,14.0,0.0,F,0-17,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1000002,P00285442,16,C,4+,8,0.0,0.0,M,55+,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,1000003,P00193542,15,A,3,1,2.0,0.0,M,26-35,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,1000004,P00184942,7,B,2,1,8.0,17.0,M,46-50,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,1000004,P00346142,7,B,2,1,15.0,0.0,M,46-50,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,1000004,P0097242,7,B,2,1,16.0,0.0,M,46-50,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,1000005,P00274942,20,A,1,8,0.0,0.0,M,26-35,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [52]:
dataset_ohe.shape

(537577, 25)